Функции проверки сгенерированного мыла.

Рассчет индекса glb и проверка безопасности.

In [1]:
def count_glb(mol):
    glb = { 'C(=O)O'                : 2.1, 
            'O'                     : 1.9, 
            'C'                     : -0.5, 
            'N'                     : 9.4, 
            'OS(=O)(=O)[O-][Na+]'   : 38.7, 
            'C(=O)[O-][Na+]'        : 19.1, 
            'C(=O)[O-][K+]'         : 21.1 }
    return sum([glb[element] for element in mol])

def check_danger(mol, strmol):
    for i in range(len(mol)):
        if (mol[i] == 'N' or mol[i] == 'O') and mol[i+1] == 'O':
            return True
    return False

Функция генерации. Входные аргументы: первые два вещества для скрещивания (Адам и Ева), группы головных и хвостовых элементов, целевой уровень glb, максимальное число итераций и уровень отсева популяции.

In [2]:
from Queue import Queue
import numpy as np
import operator

def gen_alg(adam, eve, groups_head, groups_tail, target_glb = 13, max_iters = 1000, crop = 300):
    def count_diff(mol):
        return abs(target_glb - count_glb(mol))
    all_mols = { ''.join(adam): count_diff(adam), ''.join(eve): count_diff(eve) }
    mol_molgr = { ''.join(adam): adam, ''.join(eve): eve }
    tek_iter = 0
    mol_queue = Queue()
    mol_queue.put(adam)
    mol_queue.put(eve)
    def mutate(mol_gr):
        new_mol = mol_gr[:]
        indm = np.random.randint(len(mol_gr)-1) + 1
        new_mol[indm] = groups_head[np.random.randint(len(groups_head))] \
                        if indm == len(mol_gr) - 1 else \
                        groups_tail[np.random.randint(len(groups_tail))]
        indfrag1 = np.random.randint(len(mol_gr)-1) + 1
        indfrag2 = np.random.randint(len(mol_gr)-1) + 1
        ind2frag1 = np.random.randint(len(mol_gr)-indfrag1) + indfrag1
        ind2frag2 = np.random.randint(len(mol_gr)-indfrag2) + indfrag2
        if (ind2frag1 == len(mol_gr)):
            ind2frag2 = ind2frag1
        result = [mol_gr[:], mol_gr[:], new_mol[:], new_mol[:]]
        for i in range(indfrag1, ind2frag1): result[1][i] = new_mol[i]
        for i in range(indfrag2, ind2frag2): result[2][i] = new_mol[i]
        for mol in result[1:]:
            all_mols[''.join(mol)] = count_diff(mol)
            mol_molgr[''.join(mol)] = mol
            mol_queue.put(mol)
    while (tek_iter < max_iters and not mol_queue.empty()):
        mutate(mol_queue.get())
        if len(all_mols) > crop:
            sorted_mols = sorted(all_mols.items(), key =operator.itemgetter(1))
            all_mols = { sorted_mols[i][0] : sorted_mols[i][1] for i in range(len(all_mols)/4)}
            mol_queue = Queue()
            for mol in all_mols:
                mol_queue.put(mol_molgr[mol])
        tek_iter += 1
    return sorted(all_mols.items(), key =operator.itemgetter(1))

Тестирование (получилось обычное натриевое мыло)

In [3]:
groups_head = ['C(=O)O', 'O', 'OS(=O)(=O)[O-][Na+]', 'C(=O)[O-][Na+]', 'C(=O)[O-][K+]']
groups_tail = ['C', 'O', 'N']

lauretsulphat = ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'C', 'OS(=O)(=O)[O-][Na+]']
stearat_natria = ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C(=O)[O-][Na+]']

gen_alg(lauretsulphat, stearat_natria, groups_head, groups_tail, max_iters = 20000)

[('CCCCCCCCOCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCOCCCCCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCOCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCCCCOCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCOCCCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCCCCCCOC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('COCCCCCCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCCCCCOCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCOCCCCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCCOCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCCCOCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCOCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCCCCCCOCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCOCCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCCCCOCCCCCCCCCCC(=O)[O-][Na+]', 1.7763568394002505e-15),
 ('CCCOCCCCCNCOCOOCCO', 0.09999999999999787),
 ('CCCOCCCCCNOOCCOCCO', 